In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import imblearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime
import random

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [2]:
from comet_ml import Experiment, Optimizer
from tabular_hypernet import HypernetworkPCA, TrainingModes, Hypernetwork
from tabular_hypernet.modules import SimpleNetwork
from tabular_hypernet.training_utils import train_slow_step, train_model, train_carthesian
from tabular_hypernet.interfaces import HypernetworkSklearnInterface, SimpleSklearnInterface

In [3]:
import pyhopper

In [4]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [5]:
from hypertab_benchmark_utils import *

# GLOBALS

In [6]:
DEVICE="cpu"#"cuda:0"

In [7]:
GS_METRIC = "balanced_accuracy"

In [8]:
SEED = 42

In [9]:
TEST_RUN = True

os.environ["HYPERTAB_TEST_RUN"] = str(TEST_RUN)

In [10]:
DATA = "Hill-Valley-with-noise"

# Load data

In [11]:
set_seed(SEED)

In [12]:
_X, _y = get_data(DATA)

train (606, 100) 2
test (606, 100) 2


In [13]:
n_classes = get_n_classes(_X, _y)
n_features = get_n_features(_X, _y)

n_classes 2
n_features 100


In [14]:
get_each_class_counts(_X, _y)

class counts {0: 606, 1: 606}


{0: 606, 1: 606}

# Split

In [15]:
X_train, X_test, y_train, y_test = initial_split(_X, _y)

X = X_train, X_test
y = y_train, y_test

In [16]:
n_classes = get_n_classes(X_train, y_train)
n_features = get_n_features(X_train, y_train)

n_classes 2
n_features 100


In [17]:
get_each_class_counts(X_train, y_train)

class counts {0: 307, 1: 299}


{0: 307, 1: 299}

In [18]:
eval_train_max_size, train_max_size = get_eval_and_benchmark_size(X_train=X_train)

eval_max_size 606
train_max_size 454


# TRAIN MODELS

## Common hyperparams

In [19]:
class CommonHyperparams:
    lr = pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1])
    batch_size = pyhopper.int(32, 512, 32, 32)
    ht_mask_size = pyhopper.choice([2, 5, 10, 20, 50, 90])
    ht_target_size = pyhopper.choice([5, 10, 20, 50])
    ht_mask_no = pyhopper.choice([50, 70, 100, 150, 200, 300])
    ht_epochs = pyhopper.choice([100, 200])
    

## NODE

In [ ]:
set_seed(SEED)

node_fn=get_parametrized_node_fn(X_train=X_train, n_classes=n_classes, n_features=n_features, DEVICE=DEVICE)

#### Tune hyperparams

In [ ]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 1024, power_of=2),
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
    'batch_size': pyhopper.choice([32, 64, 128]),
}

node_best, node_history = pyhopper_best_params(
    model_fn=node_fn,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time="180m"
)

node_best

#### Use best hyperparams

In [ ]:
%%capture --no-stdout

node_results = test_model(
    model_fn=node_fn(**node_best),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

In [ ]:
print_mean_std_max(node_results, DATA)

## Dropout Neural network

#### Find Hyperparams

### Dropout 1 layer

In [ ]:
set_seed(SEED)

network_fn1=get_parametrized_dropout_net1(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

In [ ]:
param_grid = {
                "epochs": CommonHyperparams.ht_epochs,
                "lr": CommonHyperparams.lr,
                "batch_size": CommonHyperparams.batch_size,
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn1_best_params, nn_fn1_history = pyhopper_best_params(
    model_fn=network_fn1,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time="60m",
)

nn_fn1_best_params

In [ ]:
nn1_results = test_model(
    model_fn=network_fn1(**nn_fn1_best_params),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

In [ ]:
print_mean_std_max(nn1_results, DATA)

### Dropout 2 layers

In [ ]:
set_seed(SEED)

network_fn2=get_parametrized_dropout_net2(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

In [ ]:
param_grid = {
                "epochs": CommonHyperparams.ht_epochs,
                "lr": CommonHyperparams.lr,
                "batch_size": CommonHyperparams.batch_size,
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
             }

nn_fn2_best_params, nn_fn2_history = pyhopper_best_params(
    model_fn=network_fn2,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time="60m",
)
nn_fn2_best_params

In [ ]:
nn2_results = test_model(
    model_fn=network_fn2(**nn_fn2_best_params),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

In [ ]:
print_mean_std_max(nn2_results, DATA)

### Dropout 3 layers

In [ ]:
set_seed(SEED)

network_fn3=get_parametrized_dropout_net3(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

In [ ]:
param_grid = {
                "epochs": CommonHyperparams.ht_epochs,
                "lr": CommonHyperparams.lr,
                "batch_size": CommonHyperparams.batch_size,
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
             }

nn_fn3_best_params, nn_fn3_history = pyhopper_best_params(
    model_fn=network_fn3,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time="60m",
)

nn_fn3_best_params

In [ ]:
nn3_results = test_model(
    model_fn=network_fn3(**nn_fn3_best_params),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

In [ ]:
print_mean_std_max(nn3_results, DATA)

### Hypernetwork

In [ ]:
set_seed(SEED)

network_hp_fn=get_parametrized_hypertab_fn(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

#### Find hyperparams

In [ ]:
param_grid = {
    "epochs": CommonHyperparams.ht_epochs,
    "masks_no": CommonHyperparams.ht_mask_no,
    "mask_size": CommonHyperparams.ht_mask_size,
    "target_size": CommonHyperparams.ht_target_size,
    "lr": CommonHyperparams.lr,
    "batch_size": CommonHyperparams.batch_size,
}

hp_best_params, hp_history = pyhopper_best_params(
    model_fn=network_hp_fn, 
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time="60m",
)

hp_best_params

In [ ]:
hp_best_params

#### Train using the best hyperparams

In [ ]:
epochs = hp_best_params['epochs']
masks_no = hp_best_params['masks_no']
mask_size = hp_best_params['mask_size']
target_size = hp_best_params['target_size']
batch_size = hp_best_params['batch_size']
lr = hp_best_params['lr']


hyper_results = test_model(
    model_fn=network_hp_fn(**hp_best_params, verbose=True),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

In [ ]:
print_mean_std_max(hyper_results, DATA)

## Random Subspace

In [20]:
set_seed(SEED)

get_bagged_fn=get_parametrized_bagged_fn()

#### Tune hyperparams

In [21]:
param_grid = {
    "first_hidden_layer": CommonHyperparams.ht_mask_size,
    "second_hidden_layer": CommonHyperparams.ht_target_size,
    "batch_size": CommonHyperparams.batch_size,
    "learning_rate_init": CommonHyperparams.lr, 
    "max_iter": CommonHyperparams.ht_epochs
}

In [22]:
bagged_best, bagged_history = pyhopper_best_params(
    model_fn=get_bagged_fn, 
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device='cpu',
    time="30m"
)

THIS IS TEST RUN
| DEVICE: cpu
| model_fn bagged_fn

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'first_hidden_layer': 2, 'second_hidden_layer': 5, 'batch_size': 32, 'learning_rate_init': 3e-05, 'max_iter': 100}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

params {'first_hidden_layer': 10, 'second_hidden_layer': 50, 'batch_size': 160, 'learning_rate_init': 0.003, 'max_iter': 100}
iter 1 of 1 X_train shape torch.Size([454, 100])


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

params {'first_hidden_layer': 20, 'second_hidden_layer': 50, 'batch_size': 224, 'learning_rate_init': 0.3, 'max_iter': 200}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'first_hidden_layer': 90, 'second_hidden_layer': 50, 'batch_size': 288, 'learning_rate_init': 0.0003, 'max_iter': 200}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization has

================= Summary =================
Mode              : Best f : Steps : Time  
----------------  : ----   : ----  : ----  
Initial solution  : 48.31  : 1     : 2.44 s
Random seeding    : 62.86  : 23    : 28 s  
Local sampling    : 63.59  : 16    : 26 s  
----------------  : ----   : ----  : ----  
Total             : 63.59  : 40    : 56 s  
Hill-Valley-with-noise_bagged_fn_{'first_hidden_layer': 20, 'second_hidden_layer': 50, 'batch_size': 288, 'learning_rate_init': 0.003, 'max_iter': 200}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Use best hyperparams

In [23]:
bagging_results = test_model(
    model_fn=get_bagged_fn(**bagged_best),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


In [24]:
print_mean_std_max(bagging_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 52.78 ~ nan (max: 52.78)


## Ensembles

In [20]:
set_seed(SEED)

get_ensembles=get_parametrized_ensemble_fn()

#### Tune

In [21]:
param_grid = {
    "n_models": CommonHyperparams.ht_mask_no,
    "first_hidden_layer": CommonHyperparams.ht_mask_size,
    "second_hidden_layer": CommonHyperparams.ht_target_size,
    "batch_size": CommonHyperparams.batch_size,
    "learning_rate_init": CommonHyperparams.lr, 
    "max_iter": CommonHyperparams.ht_epochs
}

In [22]:
ensemble_best, ensemble_history = pyhopper_best_params(
    model_fn=get_ensembles, 
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device='cpu',
    time="30m"
)

THIS IS TEST RUN
| DEVICE: cpu
| model_fn ensemble_fn

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'n_models': 50, 'first_hidden_layer': 2, 'second_hidden_layer': 5, 'batch_size': 32, 'learning_rate_init': 3e-05, 'max_iter': 100}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

params {'n_models': 150, 'first_hidden_layer': 50, 'second_hidden_layer': 5, 'batch_size': 64, 'learning_rate_init': 3e-05, 'max_iter': 100}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

==================== Summary ===================
Mode              : Best f : Steps : Time       
----------------  : ----   : ----  : ----       
Initial solution  : 50     : 1     : 14.58 s    
Random seeding    : 50.07  : 1     : 01:58 (m:s)
----------------  : ----   : ----  : ----       
Total             : 50.07  : 2     : 02:13 (m:s)
Hill-Valley-with-noise_ensemble_fn_{'n_models': 150, 'first_hidden_layer': 50, 'second_hidden_layer': 5, 'batch_size': 64, 'learning_rate_init': 3e-05, 'max_iter': 100}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Use best

In [23]:
ensemble_results = test_model(
    model_fn=get_ensembles(**ensemble_best),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization has

In [24]:
print_mean_std_max(ensemble_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 53.73 ~ nan (max: 53.73)


## XGBoost

In [25]:
set_seed(SEED)

get_xgboost = get_parametrized_xgboost_fn(seed=SEED)

#### Hyperparam tuning

In [26]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbt_best1, xgbt_history1 = pyhopper_best_params(
    model_fn=get_xgboost, 
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device='cpu',
    time="30m"
)

THIS IS TEST RUN
| DEVICE: cpu
| model_fn get_xgboost

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'n_estimators': 1550, 'max_depth': 2, 'learning_rate': 0.0010000000000000002, 'min_child_weight': 1, 'gamma': 0}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 2650, 'max_depth': 2, 'learning_rate': 0.0010356965883821555, 'min_child_weight': 2, 'gamma': 0.001}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.00044699505298341426, 'min_child_weight': 16, 'gamma': 0.001}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 2450, 'max_depth': 10, 'learning_rate': 0.01279072770525937, 'min_child_weight': 8, 'gamma': 0.001}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 900, 'max_depth': 10, 'learning_rate': 1.535549330745739e-05, 'min_child_weight': 16, 'gamma': 1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 800, 'max_depth': 10, 'learning_rate': 0.003125992257816266, 'min_child_weight

In [27]:
param_grid = {
                'subsample': pyhopper.choice([0.5, 0.6, 0.7, 0.8, 0.9, 1]),
                'reg_lambda': pyhopper.float(1e-5, 10, init=0, log=True),
                'reg_alpha': pyhopper.float(1e-5, 10, init=0, log=True),
             }


xgbt_best2, xgbt_history2 = pyhopper_best_params(
    model_fn=get_xgboost,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device='cpu',
    time="90m",
    default_params=xgbt_best1
)

THIS IS TEST RUN
| DEVICE: cpu
| model_fn get_xgboost

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'subsample': 0.5, 'reg_lambda': 0, 'reg_alpha': 0}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'subsample': 0.8, 'reg_lambda': 4.606147286266237, 'reg_alpha': 0.0001485087677600902}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'subsample': 1, 'reg_lambda': 0.00023489918564694424, 'reg_alpha': 0.16060909745261123}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'subsample': 0.7, 'reg_lambda': 0.10563166910883336, 'reg_alpha': 0.5012620066117468}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'subsample': 0.6, 'reg_lambda': 1.3438009636702366, 'reg_alpha': 0.0001485087677600902}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'subsample': 0.6, 'reg_lambda': 0.22334801124187198, 'reg_alpha': 0.0001485087677600902}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'subsample': 0.6, 'reg_lambda': 0.16287023051097393, 'reg_alpha': 0.0001485087677600902}
iter 1 of 1 X_train shape torch.Size

#### Best Params

In [28]:
xgboost_best = {**xgbt_best1, **xgbt_best2}

In [29]:
xgb_dframe = test_model(
    model_fn=get_xgboost(**xgboost_best),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


In [30]:
print_mean_std_max(xgb_dframe, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 55.83 ~ nan (max: 55.83)


## Random forest

In [31]:
set_seed(SEED)

get_rf = get_parametrized_rf_fn(seed=SEED)

#### Find hyperparams

In [32]:
param_grid = {
    'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
    'max_features': pyhopper.choice([None, 'sqrt', 0.2, 0.3, 0.5, 0.7]),
    'criterion' : pyhopper.choice(['gini', 'entropy']),
    'max_depth': pyhopper.choice([None, 2, 4, 8, 16]),
 }

rf_best, rf_history = pyhopper_best_params(
    model_fn=get_rf,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device='cpu',
    time="60m",
)

rf_best

THIS IS TEST RUN
| DEVICE: cpu
| model_fn get_rf

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'n_estimators': 1550, 'max_features': None, 'criterion': 'gini', 'max_depth': None}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 450, 'max_features': 0.5, 'criterion': 'gini', 'max_depth': 16}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 450, 'max_features': 0.3, 'criterion': 'gini', 'max_depth': 16}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 50, 'max_features': 0.5, 'criterion': 'gini', 'max_depth': 4}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 50, 'max_features': 0.3, 'criterion': 'gini', 'max_depth': 4}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 50, 'max_features': None, 'criterion': 'gini', 'max_depth': 4}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'n_estimators': 700, 'max_features': None, 'criterion': 'gini', 'max_depth': 4}
iter 1 of 1 X_train shape torch.Size([454, 100])
params

{'n_estimators': 50, 'max_features': None, 'criterion': 'gini', 'max_depth': 4}

#### Use best params

In [33]:
rf_dframe = test_model(
    model_fn=get_rf(**rf_best), 
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


In [34]:
print_mean_std_max(rf_dframe, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 52.37 ~ nan (max: 52.37)


# Collect analytics

In [ ]:
hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [ ]:
d = {}

In [ ]:
d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [ ]:
all_models_df=pd.concat(d, axis=0)
all_models_df

In [ ]:
os.environ['COMET_KEY']

In [ ]:
all_models_df.to_csv(f"{DATA}_metrics.csv")

In [ ]:
exp = Experiment(os.environ.get("COMET_KEY"), 'hypernet-uci-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune2{DATA}")
exp.log_table(f"{DATA}_metrics.csv", all_models_df)

### Replace some data in existing

In [ ]:
# tmp_df = pd.concat(d, axis=0)
# tmp_df = tmp_df.reset_index()
# tmp_df = tmp_df.rename(columns={tmp_df.columns[0]: DATA})

# tmp_df

In [ ]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")
# all_models_df = all_models_df.rename(columns={all_models_df.columns[0]: DATA})
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'Hypernet'].index)
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'HypernetPCA'].index)

# all_models_df

In [ ]:
# all_models_df = pd.concat([all_models_df, tmp_df])
# all_models_df

In [ ]:
f"{DATA}_metrics.csv"

In [ ]:
all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

In [ ]:
all_models_df.reset_index()

In [ ]:
# all_models_df = all_models_df.drop(all_models_df.columns[0], axis=1)
# all_models_df

In [ ]:
tmp = all_models_df

In [ ]:
tmp = tmp.rename(columns={tmp.columns[0]: DATA})

In [ ]:
tmp[tmp['Class'] == "balanced_accuracy"]